In [70]:
import json
from collections import namedtuple

Plan = namedtuple("Plan", "context actions")

#goldpath_file = "../data/goldsequences-0-1-2-3-4-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29.json"
goldpath_file = "../data/goldsequences-0.json"

with open(goldpath_file) as file:
    data = json.load(file)




In [71]:
task_example = data['0']['goldActionSequences'][0] # variation zero
print(task_example['taskDescription'])

plan = task_example['path']


Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


In [89]:
#print(task_example['taskDescription'].split('.')[0])
#print(plan[0]['freelook'])

goal = task_example['taskDescription'].split('.')[0]
print(f"----------- goal: {goal} -------------")
actions = []
for idx, trajectory in enumerate(plan):
    action = trajectory['action']
    actions.append(action)
    state = trajectory['freelook']
    new_obs = trajectory['observation']
    inventory = trajectory['inventory']
    #if action != 'look around':
    print("--------------------------")
    print(f"Turn {idx}")
    print(f"State: {state}")
    print(f"inventory: {inventory}")
    print(f"Action: {action}")
    print(f"New obs: {new_obs}")


----------- goal: Your task is to boil water -------------
--------------------------
Turn 0
State: This room is called the hallway. In it, you see: 
	a picture
	a substance called air
	the agent
You also see:
	A door to the green house (that is open)
	A door to the living room (that is open)
	A door to the art studio (that is open)
	A door to the kitchen (that is open)
	A door to the bedroom (that is open)
	A door to the workshop (that is open)

inventory: In your inventory, you see:
	an orange

Action: look around
New obs: This room is called the hallway. In it, you see: 
	a picture
	a substance called air
	the agent
You also see:
	A door to the green house (that is open)
	A door to the living room (that is open)
	A door to the art studio (that is open)
	A door to the kitchen (that is open)
	A door to the bedroom (that is open)
	A door to the workshop (that is open)
--------------------------
Turn 1
State: This room is called the hallway. In it, you see: 
	a picture
	a substance call

In [ ]:
"""
! if your task is to boil an water AND you are not in the kitchen
    then
! go to the kitchen (you should see a door to the kitchen)
    + open door to the kitchen
    + go to the kitchen
! pick up metal pot (you should are in the kitchen)
    + open cupboard
    + pick up metal pot
+ pick up thermometer
! fill metal pot with water (must have metal pot in your inventory and you should see a sink)
    + pour metal pot into sink
    + move metal pot to sink
    + activate sink
    + deactivate sink
    + pick up metal pot
! boil water (You should have metal pot in your inventory with water and a thermometer, and you should see a stove)
    + deactivate stove
    + move metal pot to stove
    + activate stove
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
    + examine substance in metal pot
    + use thermometer in inventory on substance in metal pot
"""

In [87]:
# NLI test
"""
This room is called the kitchen. In it, you see: a substance called soapa painting a counter. On the counter is: a bowl (containing a banana, a potato, a red apple, an orange), a drawer.a sink, which is turned off. In the sink is: nothing.a table. On the table is: a glass cup (containing nothing).a chair. On the chair is: nothing.a freezer. The freezer door is closed. a lightera stopwatch, which is deactivated. a fridge. The fridge door is closed. a substance called aira cupboard. The cupboard door is open. In the cupboard is: a tin cup (containing nothing), a ceramic cup (containing nothing), a drawer.a oven, which is turned off. The oven door is closed. You also see:A door to the outside (that is open)A door to the bathroom (that is open)A door to the hallway (that is open) <SEP> you should see a closed door to the kitchen
"""

['look around',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'pick up thermometer',
 'open cupboard',
 'pick up metal pot',
 'look around',
 'move metal pot to sink',
 'activate sink',
 'deactivate sink',
 'pick up metal pot',
 'focus on substance in metal pot',
 'pour metal pot into metal pot',
 'pick up metal pot',
 'move metal pot to stove',
 'activate stove',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in

In [85]:
your goal is to boil water and (plan context) you are in the kitchen and you see an thermometer then
pick up thermometer
open cupboard
pick up metal pot
move metal pot to sink

SyntaxError: invalid syntax (2811270517.py, line 1)

In [83]:
thpick up OBJ

if you see the OBJ

['look around',
 'open door to kitchen',
 'go to kitchen',
 'look around',
 'pick up thermometer',
 'open cupboard',
 'pick up metal pot',
 'look around',
 'move metal pot to sink',
 'activate sink',
 'deactivate sink',
 'pick up metal pot',
 'focus on substance in metal pot',
 'pour metal pot into metal pot',
 'pick up metal pot',
 'move metal pot to stove',
 'activate stove',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in inventory on substance in metal pot',
 'examine substance in metal pot',
 'use thermometer in

In [81]:
len([a for a in actions if a != 'look around'])

37

In [66]:
if you are in {location} and you see a {obj}
then

18

In [ ]:
# comparar freelook do estado atual com o anterior
# o que mudar deve continuar no plan context
# o que não mudar, da pra remover no plan context, pois a ação não alterou o estado
# o plan context terá apenas